In [5]:
import os
import json
import requests
from starter import *
from bs4 import BeautifulSoup

url = "https://170-leaderboard.vercel.app/team/discordggTYsy64VcWT"
page = requests.get(url)
data = BeautifulSoup(page.content, 'html.parser')

def get_team_json_table():
    json_table = data.find_all('script')[9].string
    json_table = json.loads(json_table)
    json_table = json_table["props"]["pageProps"]["data"]
    return json_table

def get_result_json_table():
    json_table = data.find_all('script')[9].string
    json_table = json.loads(json_table)
    json_table = json_table["props"]["pageProps"]["sortedData"]
    return json_table

def expand_input(name):
    if name.startswith('large'):
        return ('large', int(name[5:-3]))
    elif name.startswith('medium'):
        return ('medium', int(name[6:-3]))
    elif name.startswith('small'):
        return ('small', int(name[5:-3]))

class Result:
    size = None
    test = None
    rank = None
    true_rank = None
    best_score = None
    submission_score = None
    local_score = None
    local_file = None
    url = None
    notes = "_"

    def __init__(self, name, rank, best_score, submission_score):
        self.size, self.test = expand_input(name)
        self.rank = rank
        self.true_rank = rank
        self.best_score = best_score
        self.submission_score = submission_score
        self.url = f"https://170-leaderboard.vercel.app/input/{self.size}/{self.test}"

    def __str__(self):
        return f"{self.size} {self.test} {self.rank} {self.best_score} {self.submission_score} {self.local_score} {self.local_file} {self.url} {self.notes}"
    
    def __repr__(self):
        return self.__str__()

with open("../queue.txt", "w") as f:
    f.write("")

team_json_table = get_team_json_table()
table = dict()
for key in team_json_table:
    table[key] = (team_json_table[key]["score"], team_json_table[key]["rank"])

sorted_table = []
for key in table:
    sorted_table.append(Result(key, table[key][1], table[key][0], table[key][0]))
sorted_table.sort(key=lambda x: x.rank, reverse=True)

for result in sorted_table:
    if result.rank != 1:
        page = requests.get(result.url)
        data = BeautifulSoup(page.content, 'html.parser')
        result_json_table = get_result_json_table()
        result.best_score = result_json_table[0]["score"]

best_locals = dict()

for size in ['small', 'medium', 'large']:
    for test in range(1, 261):
        best_file = 'inf.in'
        best_score = float('inf')
        folder = '../tests/' + size + '/' + size + str(test)
        G = read_input(folder + '/graph.in')
        for filename in os.listdir(folder):
            if filename.endswith('.out') and float((filename.split('_')[0]).split('.')[0]) <= \
                    float((best_file.split('_')[0]).split('.')[0]):
                read_output(G, folder + '/' + filename)
                validate_output(G)
                score_G = score(G)
                if round(score_G) != float((filename.split('_')[0]).split('.')[0]):
                    print('Score mismatch, file: ' + folder + '/' + filename + ', score: ' + str(score_G))
                if score_G < best_score:
                    best_score = score_G
                    best_file = filename
        best_locals[(size, str(test))] = (best_score, best_file[:-4])
        
for result in sorted_table:
    result.local_score, result.local_file = best_locals[(result.size, str(result.test))]
    if result.submission_score == result.best_score:
        result.notes = "published_rank_1"
    elif result.local_score <= result.best_score:
        result.true_rank = 1
        result.notes = "sleeper"
    elif result.local_score > result.submission_score:
        result.notes = "missing_local"

sorted_table.sort(key=lambda x: min(x.submission_score, x.local_score) - x.best_score, reverse=True)

for result in sorted_table:
    print(result, min(result.submission_score, result.local_score) - result.best_score, file=open("../queue.txt", "a"))

true_avg_rank = 0
for result in sorted_table:
    true_avg_rank += result.true_rank
true_avg_rank /= len(sorted_table)
print(true_avg_rank)

1.141025641025641
